In [187]:
import pandas as pd
import glob
import re
from zipfile import ZipFile

In [188]:
def init_dataframe_from_zip(path, index_col):
    df_master = pd.DataFrame()
    flag = False
    files_in_folder = glob.glob(path)
    for filename in files_in_folder:    
        zip_file = ZipFile(filename)
        for text_file in zip_file.infolist():
            if text_file.filename.endswith('.csv'):
                df = pd.read_csv(zip_file.open(text_file.filename),header=0,index_col=[index_col])
            if not flag:
                df_master = df
                flag = True
            else:
                df_master = pd.concat([df_master, df])
    return df_master

csse_covid_19_daily_reports_us = init_dataframe_from_zip(r'**csv_files/csse_covid_19_daily_reports_us.zip', 'UID')
csse_covid_19_daily_reports_us.head()


,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,ISO3,Testing_Rate,Hospitalization_Rate
UID,,,,,,,,,,,,,,,,,
84000001.0,Alabama,US,2022-01-03 02:33:49,32.3182,-86.9023,909541,16455,NaN,NaN,1.0,18550.003722,6562722.0,NaN,1.809154,USA,133846.102074,NaN
84000002.0,Alaska,US,2022-01-03 02:33:49,61.3707,-152.4044,157169,978,NaN,NaN,2.0,21484.529318,3685702.0,NaN,0.622260,USA,503824.371706,NaN
16.0,American Samoa,US,2022-01-03 02:33:49,-14.2710,-170.1320,10,0,NaN,NaN,60.0,17.972359,2140.0,NaN,0.000000,ASM,3846.084722,NaN
84000004.0,Arizona,US,2022-01-03 02:33:49,33.7298,-111.4312,1390409,24355,NaN,NaN,4.0,19102.391259,15679982.0,NaN,1.751643,USA,215422.333359,NaN
84000005.0,Arkansas,US,2022-01-03 02:33:49,34.9697,-92.3731,572822,9196,NaN,NaN,5.0,18981.418276,4441472.0,NaN,1.605385,USA,147175.628371,NaN


In [193]:
def init_state_social_distancing_actions(path):
    df_master = pd.DataFrame()
    flag = False
    files_in_folder = glob.glob(path)
    for filename in files_in_folder:    
        zip_file = ZipFile(filename)
        for text_file in zip_file.infolist():
            # if not text_file.filename.startswith('__MACOSX/'):
            if text_file.filename.endswith('.csv'):
                date = re.search('\d*-\d*-\d*', text_file.filename)[0]
                if date == "20201-06-01":
                    date = "2021-06-01"
                date_time_value = pd.to_datetime(date)
                df = pd.read_csv(zip_file.open(text_file.filename), sep=",", header=0)
                df["Date"] = date_time_value
                df.rename(columns = {'Unnamed: 0':'State'}, inplace = True)
                df.drop((df[df.State.isin(["United States"])].index) | (df[df.State.isnull()].index), inplace=True)
            if not flag:
                df_master = df
                flag = True
            else:
                df_master = pd.concat([df_master, df])
    df_master.set_index(["Date", "State"], inplace=True)
    df_master.sort_index(inplace=True)
    return df_master

state_social_distancing_actions = init_state_social_distancing_actions(r'**csv_files/state_social_distancing_actions.zip')

In [194]:
def clean_state_social_distancing_actions(df):
  # TODO: Remove cells we don't want
  df = df.drop(columns=["Primary Election Postponement"])
  return df

cleaned_state_social_distancing_actions = clean_state_social_distancing_actions(state_social_distancing_actions)
cleaned_state_social_distancing_actions

State Is Easing Social Distancing Measures  \
Date       State                                                      
2020-06-04 Alabama                                              Yes   
           Alaska                                               Yes   
           Arizona                                              Yes   
           Arkansas                                             Yes   
           California                                           Yes   
...                                                             ...   
2021-11-09 Virginia                                             NaN   
           Washington                                           NaN   
           West Virginia                                        NaN   
           Wisconsin                                            NaN   
           Wyoming                                              NaN   

                         Stay at Home Order  \
Date       State                              
2020-06-04 Alabama                   Lifted   
           Alaska                    Lifted   
           Arizona                   Lifted   
           Arkansas                       -   
           California             Statewide   
...                                     ...   
2021-11-09 Virginia                     NaN   
           Washington                   NaN   
           West Virginia                NaN   
           Wisconsin                    NaN   
           Wyoming                      NaN   

                         Mandatory Quarantine for Travelers  \
Date       State                                              
2020-06-04 Alabama                                        -   
           Alaska                             All Travelers   
           Arizona                                   Lifted   
           Arkansas                     From Certain States   
           California                                     -   
...                                                     ...   
2021-11-09 Virginia                                     NaN   
           Washington                                   NaN   
           West Virginia                                NaN   
           Wisconsin                                    NaN   
           Wyoming                                      NaN   

                                            Non-Essential Business Closures  \
Date       State                                                              
2020-06-04 Alabama        All Non-Essential Businesses Permitted to Reop...   
           Alaska          All Non-Essential Businesses Permitted to Reopen   
           Arizona        All Non-Essential Businesses Permitted to Reop...   
           Arkansas                                                       -   
           California     Some Non-Essential Businesses Permitted to Reo...   
...                                                                     ...   
2021-11-09 Virginia                                                     NaN   
           Washington                                                   NaN   
           West Virginia                                                NaN   
           Wisconsin                                                    NaN   
           Wyoming                                                      NaN   

                               Large Gatherings Ban  \
Date       State                                      
2020-06-04 Alabama                           Lifted   
           Alaska                            Lifted   
           Arizona                           Lifted   
           Arkansas           >10 People Prohibited   
           California     All Gatherings Prohibited   
...                                             ...   
2021-11-09 Virginia                             NaN   
           Washington                           NaN   
           West Virginia                        NaN   
           Wisconsin                

In [195]:
california_data = cleaned_state_social_distancing_actions[cleaned_state_social_distancing_actions.index.get_level_values('State').isin(['California'])]
california_data

,,State Is Easing Social Distancing Measures,Stay at Home Order,Mandatory Quarantine for Travelers,Non-Essential Business Closures,Large Gatherings Ban,School Closures,Restaurant Limits,Emergency Declaration,Face Covering Requirement,Status of Reopening,Bar Closures,Bar Closures*,Statewide Face Mask Requirement
Date,State,,,,,,,,,,,,,
2020-06-04,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-05,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-08,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-12,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-15,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-05,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reopened,NaN,NaN,Unvaccinated People Only
2021-10-12,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reopened,NaN,NaN,Unvaccinated People Only
2021-10-19,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reopened,NaN,NaN,Unvaccinated People Only
